In [1]:
% Exercise 2 a) load data
data = load('Assignment4 Data-20221224/Assignment4_Data/A3P14_Stock_data.mat')

data =

  scalar structure containing the fields:

    Stock_data =

        11.026    25.120
        11.243    25.370
        11.250    25.250
        11.317    25.310
        11.113    25.258
        11.300    26.040
        11.363    26.890
        11.337    27.020
        11.434    26.840
        11.365    26.870
        11.465    27.230
        11.420    27.080
        11.449    27.340
        11.488    27.245
        11.439    27.620
        11.441    27.850
        11.473    27.988
        11.450    27.902
        11.508    27.810
        11.579    28.070
        11.613    28.188
        11.557    28.300
        11.596    28.070
        11.624    28.010
        11.617    27.970
        11.559    27.850
        11.520    27.910
        11.770    27.980
        11.832    28.087
        11.929    28.000
        11.919    28.820
        12.004    28.600
        12.230    28.220
        12.201    28.110
        12.301    28.550
        12.346    28.190
        12.446    28.300
      

In [14]:
% b) write function
function a_vector = seq_LS(data_col,order) % return_value = function_name(arguemts)
    % Using The idea of Sequential Least Squares to avoid big matrix inversions and multiplications after 
    % lecture OSAP_02a slide 115:

    % Initalizing with the idea of lecture OSAP_02a slide 118:
    a_vector = zeros(order,1);
    sigma_matrix = eye(order)*1000000;
    
    for i = (order+1):length(data_col), % until all n in data done
        h = flip(data_col((i-order):(i-1))); % what we measured in past 
        K_vector = (sigma_matrix * h) / ((1/1) + transpose(h) * sigma_matrix * h);
        sigma_matrix = (eye(order) - K_vector * transpose(h)) * sigma_matrix;
        x = data_col(i); % what we measure now/the current y
        a_vector = a_vector + K_vector * (x - transpose(h) * a_vector);    
    endfor;
    
endfunction

In [15]:
% c) Use the with 1500 samples

data_matrix = data.Stock_data; % Stock_data is the only key/field of our data
% data_matrix(2,1); % get the 2. row in the 1. columns
% data_matrix(:,2); % get the 2. column

order=3;
data_col = data_matrix(1:1500,1);
estimated_a = seq_LS(data_col, order);
estimated_a

estimated_a =

   1.020754
  -0.050099
   0.029655



a1 seems to be the most important estimator value (1.020754) because it has the highest value. Therefore, data one time step before the prediction seems to give the most relevant information. The other estimator values are around 0 for order 3. Therefore, older time steps don't seem to influence the share's course very much.

In [54]:
% d) Use the remaining samples to evaluate the estimator:

data_col_val = data_matrix(1500:size(data_matrix)(1),1);
y_pred = [];

for i = (order+1):length(data_col_val), % until all n in data done
    h = transpose(flip(data_col_val((i-order):(i-1)))); % what we measured in past 
    y_pred_n = h * estimated_a;
    y_pred = [y_pred; y_pred_n];
endfor;


In [55]:
pkg load 'image' % for immse (mean squared error)
y_true = data_col_val((order+1):size(data_col_val)(1),1);
mse_ = immse(y_pred, y_true);
mse_

mse_ = 1.9667


For each sample we have on average a difference of 1.9967 units/dollars for prediction versus true value. Thats not that bad considering that the true values range from 26.427 up to 134.18 .

In [67]:
% e) Use estimator 'a' as unit vector
a_unit = zeros(length(estimated_a),1);
a_unit(1) = 1;

y_pred_unit = [];
for i = (order+1):length(data_col_val), % until all n in data done
    h = transpose(flip(data_col_val((i-order):(i-1)))); % what we measured in past 
    y_pred_n_unit = h * a_unit;
    y_pred_unit = [y_pred_unit; y_pred_n_unit];
endfor;

In [69]:
mse_unit = immse(y_pred_unit, y_true);
mse_unit

mse_unit = 1.9479


With a MSE of 1.9497 the unit estimator is minimally better than the estimator based on the model (MSE=1.9967). However, the model based estimator comes with estimated_a = [1.020754
  -0.050099
   0.029655] $^T$ very close to the unit estimator which is a_unit = [1 0 0]$^T$ . When we would increae the training size or initalize the sigma_matrix with even higher values it could be that we even come closer to the unit vector as estimation. <br>
   Also here we can see again that only thinking about the last time step an ignoring all other time steps can lead to a good result.